# TEI Bulk Embedding Load Test

I think a good workflow:

1. Use load tests with k6 to do grid search (it handles failures well)
2. Then use my scripts to confirm we get similar performance when making async calls

Also need to include a demo of how to keep track of the order / tie a specific response to a given input record... apparently asyncio respects the ordering of requests..

Resources:

- On asyncio in Python: https://superfastpython.com/asyncio-gather/
- On `as_completed()` vs. `gather()`: https://jxnl.github.io/instructor/blog/2023/11/13/learn-async/#asyncioas_completed-handling-tasks-as-they-complete


Goal:

- Embed N chunks of data in total
- Vary the batch_size of each request, as well as the concurrency of requests
- Analyze the total time to complete, RPS, and latency to see which bs / concurrency combination is best.


In [3]:
batches = generate_batches("This is a test", bs=10, total_chunks=1000)
results = await embed(batches, concurrency_level=5, collect_results=True)

Batch Size: 10, Concurrency Level: 5, Total Time: 1.77 seconds, RPS: 11.31, Embed per sec: 565.45


In [1]:
import os
import asyncio
from dotenv import load_dotenv

from tei_bulk_embed_test import main

%load_ext autoreload
%autoreload 2

load_dotenv(override=True)

In [3]:
data = [
    {"unique_id": n, "text": f"This is a test sentence - {n}"} for n in range(50_000)
]

In [5]:
batch_size = 32
concurrency = 1000

# os.remove("embeddings.jsonl")
results = await main(data, batch_size, concurrency, filename=False)

Timing Metrics Statistics: 


total_time:
  mean: 1.321 seconds
  min: 0.298 seconds
  median: 1.404 seconds
  max: 2.007 seconds
  p90: 1.596 seconds
  p95: 1.652 seconds

tokenization_time:
  mean: 0.0201 seconds
  min: 0.0 seconds
  median: 0.018 seconds
  max: 0.071 seconds
  p90: 0.039 seconds
  p95: 0.043 seconds

queue_time:
  mean: 0.5067 seconds
  min: 0.004 seconds
  median: 0.403 seconds
  max: 0.947 seconds
  p90: 0.743 seconds
  p95: 0.896 seconds

inference_time:
  mean: 0.444 seconds
  min: 0.02 seconds
  median: 0.564 seconds
  max: 0.681 seconds
  p90: 0.574 seconds
  p95: 0.575 seconds


Total pipeline execution time (time to embed all data): 25.2797 seconds
Total number of chunks to embed: 50000
Total number of chunks embedded: 50000
Total number of chunks that hit an error: 0
Embeddings per second (completed requests): 1977.8715728430323


In [15]:
results

{'timing_statistics': {'total_time': {'mean': 1344.69184,
   'min': 54.0,
   'median': 1343.0,
   'max': 1818.0,
   'p90': 1637.0,
   'p95': 1711.0},
  'tokenization_time': {'mean': 17.05504,
   'min': 0.0,
   'median': 16.0,
   'max': 70.0,
   'p90': 32.0,
   'p95': 38.0},
  'queue_time': {'mean': 550.98144,
   'min': 4.0,
   'median': 514.0,
   'max': 907.0,
   'p90': 721.0,
   'p95': 893.0},
  'inference_time': {'mean': 404.72544,
   'min': 27.0,
   'median': 452.0,
   'max': 459.0,
   'p90': 456.0,
   'p95': 458.0}},
 'total_time': 25.407,
 'embeddings_per_second': 1967.961585389853,
 'success_metrics': {'success': 50000, 'failure': 0, 'total': 50000}}

In [ ]:
async def run_experiments():
    batch_sizes = [1, 4, 8, 16, 32]
    concurrency_levels = [1, 25, 50, 100, 205, 500, 1000]

    results = []
    for batch_size in batch_sizes[-2:]:
        for concurrency_level in concurrency_levels[-2:]:
            batches = generate_batches(
                "This is a test", bs=batch_size, total_chunks=10_000
            )
            results.append(
                await embed(
                    batches,
                    concurrency_level=concurrency_level,
                    collect_embeddings=False,
                )
            )
    return results